In [1]:
%matplotlib widget

In [2]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json

import os

In [3]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"


In [4]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

# input json name

In [5]:
file_parameters['watershed'] = 'geum_1'

In [6]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [7]:
pd.set_option('display.max_columns', 1000)

In [8]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
model_input_path = file_parameters['watershed']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [9]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [10]:
gain_epochs , rnn_epochs

(2000, 15)

# run file

In [11]:
run_num = range(len(folder))


# run_num = [0]




real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)


# Gain Training Flag

In [12]:
__GAIN_TRAINING__ = False

In [13]:
month=12
day=31
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #start = time.time()

    #if watershed == '한강_12days_test':
    #    df, times = make_dataframe_temp_12days(folder[idx], file_names[idx], colum_idx[idx], interpolate=interpolation_option[idx])
    #else:
    df, times , month, day= make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], 
                               interpolation=interpolation_option[idx],first_file_no=i, month=month, day=day)

    df_all, train_mean, train_std, df = normalize(df)


    

    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx'], ['장계_2015.xlsx', '장계_2016.xlsx', '장계_2017.xlsx', '장계_2018.xlsx', '장계_2019.xlsx', '장계_2020.xlsx']]
data/geum/자동/현도_2015.xlsx
data/geum/자동/현도_2016.xlsx
data/geum/자동/현도_2017.xlsx
data/geum/자동/현도_2018.xlsx
data/geum/자동/현도_2019.xlsx
data/geum/자동/현도_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/대청호_2015.xlsx
data/geum/자동/대청호_2016.xlsx
data/geum/자동/대청호_2017.xlsx
data/geum/자동/대청호_2018.xlsx
data/geum/자동/대청호_2019.xlsx
data/geum/자동/대청호_2020.xlsx
time range in files :  2015-01-01 00:00  ~  2020-12-31 23:00
data/geum/자동/장계_2015.xlsx
data/geum/자동/장계_2016.xlsx
data/geum/자동/장계_2017.xlsx
data/geum/자동/장계_2018.xlsx
data/geum/자동/장계_2019.xlsx
data/geum/자동/

In [15]:
print(real_df_all.shape)

(52608, 137)


In [16]:
def dataset_slice(df, train_ratio, val_ratio, test_ratio):
    total_no = df.shape[0]

    train_no = int(total_no * train_ratio)
    #
    val_no = int(total_no * (train_ratio + val_ratio))

    # val_no = int(total_no*val_ratio)
    # train_no =int(total_no * (train_ratio+val_ratio))
    #
    # val_slice = slice(0, val_no)
    # train_slice = slice(val_no, train_no)
    # test_slice = slice(train_no, None)

    train_slice = slice(0, train_no)    #0.8
    val_slice = slice(train_no, val_no)   #0.1
    test_slice = slice(val_no, None)
    # val_slice = slice(train_no, val_no)   #0.1
    # test_slice = slice(val_no, None)
#     train_slice = slice(0, train_no)  # 0.8
#     val_slice = slice(train_no, None)  # 0.1
#     test_slice = slice(val_no, None)

    train = pd.DataFrame(df[train_slice])
    val = pd.DataFrame(df[val_slice])
    test = pd.DataFrame(df[test_slice])

    test_slice2 = slice(total_no - 288, None)
    test2 = pd.DataFrame(df[test_slice2])

    #print('total_no :1111111111111111111111111 ')
    #print('total_no : ', total_no)
    #print('train : ', train.shape)
    #print('val : ', val.shape)
    #print('test : ', test.shape)


    return train, val, test, test2

In [17]:
df[0]

,nh3n,no3n,ph,cod,ec,bod,ss,do,tn,tp,wtp,chlorophylla,po4p,dtn,dtp,toc,Day sin,Day cos,Year sin,Year cos
0,0.006763,0.484728,-0.101790,0.083502,-0.287640,-0.139791,-0.166118,0.357192,-0.160689,-0.200862,-0.456726,-0.206121,-0.129355,-0.101337,-0.006470,-0.192947,1.747686e-12,1.414210,0.002127,1.413681
1,0.005774,0.484619,-0.102667,0.084437,-0.287578,-0.137465,-0.165609,0.355512,-0.161080,-0.202437,-0.456635,-0.206087,-0.129492,-0.102015,-0.007501,-0.191757,3.660245e-01,1.366022,0.003141,1.413679
2,0.004802,0.484510,-0.103531,0.085358,-0.287517,-0.135174,-0.165108,0.353857,-0.161465,-0.203987,-0.456546,-0.206054,-0.129628,-0.102683,-0.008516,-0.190587,7.071051e-01,1.224742,0.004155,1.413677
3,0.003845,0.484403,-0.104381,0.086266,-0.287457,-0.132916,-0.164614,0.352227,-0.161845,-0.205514,-0.456459,-0.206021,-0.129761,-0.103341,-0.009516,-0.189437,9.999976e-01,0.999998,0.005168,1.413674
4,0.002904,0.484297,-0.105218,0.087160,-0.287397,-0.130692,-0.164129,0.350623,-0.162218,-0.207017,-0.456372,-0.205989,-0.129891,-0.103989,-0.010500,-0.188304,1.224742e+00,0.707105,0.006182,1.413669
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52603,0.149795,0.501465,-0.174335,0.467277,-0.765332,-0.269849,0.502214,-0.109869,-0.023474,1.038615,0.405482,-0.135313,0.381685,0.042927,0.910679,0.386185,-1.366022e+00,0.366025,0.010318,1.413645
52604,0.117644,0.508780,-0.168390,0.434043,-0.772617,-0.307423,0.474602,-0.104741,-0.032294,1.058158,0.400472,-0.167388,0.344765,0.037270,0.941676,0.354350,-1.224742e+00,0.707105,0.011332,1.413638
52605,0.085441,0.516107,-0.162436,0.400754,-0.779914,-0.345058,0.446942,-0.099605,-0.041129,1.077732,0.395454,-0.199516,0.307785,0.031605,0.972723,0.322462,-9.999976e-01,0.999998,0.012346,1.413629
52606,0.053185,0.523447,-0.156473,0.367411,-0.787222,-0.382755,0.419233,-0.094461,-0.049978,1.097338,0.390428,-0.231697,0.270743,0.025930,1.003822,0.290523,-7.071051e-01,1.224742,0.013360,1.413620


# slice DF

In [18]:
# fake_df=real_df_all.iloc[:-2000, :]

In [19]:
train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.7, 0.1, 0.2)

In [20]:
val_df.shape, test_df.shape, train_df.shape

((10522, 137), (5261, 137), (36825, 137))

In [21]:
print('-------------------prediction')
print('-------------------prediction')
print('-------------------prediction')

print('real_df_all.type : ', type(real_df_all))
print('train_df.type : ', type(train_df))
print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)


-------------------prediction
-------------------prediction
-------------------prediction
real_df_all.type :  <class 'pandas.core.frame.DataFrame'>
train_df.type :  <class 'pandas.core.frame.DataFrame'>
train_df.shape :  (36825, 137) val_df.shape :  (10522, 137) test_df.shape: (5261, 137)


# target column

In [22]:

label_columns_indices = {name: i for i, name in enumerate(dfff[0])}

print("label_columns_indices:")
print(label_columns_indices)


target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}

rnn_target_column = "toc"

print('target columns : ', rnn_target_column)
num_features = dfff[0].shape[1]



target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target columns :  toc
target_col_idx :  4
out_num_features :  1


In [25]:
val_nse = {}
val_pbias = {}


WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df, test_df2=test_df2)

# if __RNN_TRAINING__:
#     if not os.path.exists('save/' + model_input_path):
#         os.makedirs('save/' + model_input_path)


idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]

indices = {name: i for i, name in enumerate(idx)}

model_path = "save/" + watershed + "models/" + model_input_path+ '/' + pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

val_nse = {}
val_pbias = {}

 # +"gru.ckpt" -- path




save model path :  save/geum/models/geum_1/toc/


In [26]:
plt.figure()
plt.plot(real_df_all.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [27]:
plt.figure()
plt.plot(test_df.iloc[:,out_features[0]])
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

## 모델 학습

In [28]:
__RNN_TRAINING__ = True
continue_train = False

In [29]:
rnn_epochs = 15
multi_linear_model = model_multi_linear(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    #training_flag=__RNN_TRAINING__, checkpoint_path="save/"+watershed+"models/multi_linear.ckpt")
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_linear.ckpt", continue_train=continue_train)
print('ELMAN Running....')
elman_model = model_elman(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"elman.ckpt", continue_train=continue_train)
print('GRU Running....')
gru_model = model_gru(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=continue_train)
print('LSTM Running....')
multi_lstm_model = model_multi_lstm(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_lstm.ckpt", continue_train=continue_train)
print('CNN Running....')
multi_conv_model = model_multi_conv(
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"multi_conv.ckpt", continue_train=continue_train)


Epoch 1/15
10/10 [==============================] - 25s 3s/step - loss: 1.1843 - mean_absolute_error: 0.8787 - nse: -1.7025 - val_loss: 3.4796 - val_mean_absolute_error: 1.0916 - val_nse: -8.8870

Epoch 00001: val_nse improved from -inf to -8.88701, saving model to save/geum/models/geum_1/toc/multi_linear.ckpt
Epoch 2/15
10/10 [==============================] - 24s 3s/step - loss: 0.7769 - mean_absolute_error: 0.7060 - nse: -0.9548 - val_loss: 4.9022 - val_mean_absolute_error: 1.1644 - val_nse: -12.8961

Epoch 00002: val_nse did not improve from -8.88701
Epoch 3/15
10/10 [==============================] - 23s 3s/step - loss: 0.5469 - mean_absolute_error: 0.5856 - nse: -0.2542 - val_loss: 6.7374 - val_mean_absolute_error: 1.2700 - val_nse: -18.1010

Epoch 00003: val_nse did not improve from -8.88701
Epoch 4/15
10/10 [==============================] - 23s 3s/step - loss: 0.4061 - mean_absolute_error: 0.5019 - nse: -0.0183 - val_loss: 8.3652 - val_mean_absolute_error: 1.3543 - val_nse: -2

Epoch 1/15
10/10 [==============================] - 26s 3s/step - loss: 0.6379 - mean_absolute_error: 0.6459 - nse: -0.5529 - val_loss: 0.2253 - val_mean_absolute_error: 0.3624 - val_nse: 0.3649

Epoch 00001: val_nse improved from -inf to 0.36491, saving model to save/geum/models/geum_1/toc/gru.ckpt
Epoch 2/15
10/10 [==============================] - 22s 2s/step - loss: 0.2296 - mean_absolute_error: 0.3736 - nse: 0.4892 - val_loss: 0.2557 - val_mean_absolute_error: 0.3791 - val_nse: 0.2812

Epoch 00002: val_nse did not improve from 0.36491
Epoch 3/15
10/10 [==============================] - 24s 3s/step - loss: 0.1638 - mean_absolute_error: 0.3110 - nse: 0.6366 - val_loss: 0.1388 - val_mean_absolute_error: 0.2792 - val_nse: 0.6095

Epoch 00003: val_nse improved from 0.36491 to 0.60954, saving model to save/geum/models/geum_1/toc/gru.ckpt
Epoch 4/15
10/10 [==============================] - 24s 3s/step - loss: 0.1182 - mean_absolute_error: 0.2643 - nse: 0.7177 - val_loss: 0.1580 - val_mea


Epoch 00015: val_nse did not improve from 0.50444
CNN Running....
Epoch 1/15
10/10 [==============================] - 25s 3s/step - loss: 0.4205 - mean_absolute_error: 0.5044 - nse: 0.0777 - val_loss: 9.4923 - val_mean_absolute_error: 1.2929 - val_nse: -26.0443

Epoch 00001: val_nse improved from -inf to -26.04429, saving model to save/geum/models/geum_1/toc/multi_conv.ckpt
Epoch 2/15
10/10 [==============================] - 22s 2s/step - loss: 0.1336 - mean_absolute_error: 0.2796 - nse: 0.6956 - val_loss: 18.7523 - val_mean_absolute_error: 1.6876 - val_nse: -52.3449

Epoch 00002: val_nse did not improve from -26.04429
Epoch 3/15
10/10 [==============================] - 22s 2s/step - loss: 0.0991 - mean_absolute_error: 0.2305 - nse: 0.7647 - val_loss: 24.2166 - val_mean_absolute_error: 1.8504 - val_nse: -67.8249

Epoch 00003: val_nse did not improve from -26.04429
Epoch 4/15
10/10 [==============================] - 24s 3s/step - loss: 0.0650 - mean_absolute_error: 0.1874 - nse: 0.8408

## core / window.py / 

In [29]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    
    print(df.shape)
    print(plot_col)
    
    width = input_width + label_width
    
    length = df.shape[0]
    length -= width
    
    inputs = []
    labels = []
   
    for i in range(0,length,24):
#         print('i = ', i)
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    
#     print('labels.mean(axis=1)')
#     print(labels.mean(axis=1).shape)
    print(inputs.shape)
    print(labels.shape)

    predictions = model(inputs)
    print(predictions.shape)
    
    predictions = predictions.numpy() * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    print('predictions.shape =', predictions.shape)
#     pred_day = predictions
    pred_day = hour_to_day_mean(predictions)
    print('pred_day.shape =', pred_day.shape)
#     label_day= labels
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
#     inputs_day = inputs_target
    inputs_day = hour_to_day_mean(inputs_target)
    
    #plt.figure(figsize=(10, 800))
    
#     input_index = np.array(range(0,length,24))
#     label_index = input_index + 24*7
    
#     print(input_index)
#     print(label_index)
    
    plt.figure()
    plt.plot(label_day[:, predict_day, :], label='label')
    plt.plot(pred_day[:, predict_day, :], label='pred')
#     plt.plot(input_index, inputs_day[:, 0, :], label='input')
#     plt.plot(label_index, label_day[:, 0, :], label='label')
#     plt.plot(label_index, pred_day[:, 0, :], label='pred')
    plt.legend()
        
        
        
#     o1 = np.mean(labels)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((labels - predictions)**2).sum()
#     nse2 = ((labels - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
    
    
    
#     print('pred_day.shape =',pred_day.shape)
#     print('label_day.shape =',label_day.shape)
    #print('label_index =', label_index)
    
    o1 = np.mean(label_day[:,predict_day,0])
#     o1 = np.mean(label_day)
#     print('labels =', labels)
#     o1 = target_mean[plot_col]
    print('o1 =', o1)
    nse1 = ((pred_day-label_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
    print('label_day[predict_day].shape =', label_day[predict_day].shape)
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
    o = label_day[:, 0, 0]
    s = pred_day[:, 0, 0]
    mean = np.mean(o)
    print('mean =', mean)
    nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
#     pbias1 = (label_day - pred_day).sum(axis=0)
#     pbias2 = (label_day).sum(axis=0)
#     pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
#     o1 = np.mean(label_day)
# #     print('labels =', labels)
#     print('o1 =', o1)
#     nse1 = ((label_day - pred_day)**2).sum()
#     nse2 = ((label_day - o1)**2).sum()
#     nse3 = 1 - (nse1/nse2)
#     print('nse3 =', nse3)
    #mean = np.mean(label_day[predict_day])
#     print(type(pred_day))
#     print(pred_day[:20, 0, 0])
#     print(label_day[:20, 0, 0])
#     o = label_day[:, 0, 0]
#     s = pred_day[:, 0, 0]
#     mean = np.mean(o)
#     print('mean =', mean)
#     nse = 1. - np.sum((o-s)**2)/np.sum((o-mean)**2)
#     print('nse =', nse)
    
    
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    
    
    return float(nse3), float(np.abs(pbias3)), pred_day, labels
    

In [30]:
test_df

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79
35800,-1.135188,-0.683542,-0.035966,-0.532714,-0.195010,-0.590213,-0.451605,-0.592412,-1.224741,-7.071045e-01,0.691402,1.262056,-1.115422,-0.602486,0.501096,-1.072646,-0.091820,0.022988,-0.507353,-0.603172,-1.224741,-7.071045e-01,0.691402,1.262056,-1.517278,-0.550392,1.562846,0.711145,0.398755,1.154331,-0.205315,0.324512,-1.224741,-7.071045e-01,0.691402,1.262056,-0.200503,-0.978858,-1.054064,0.271219,-0.027959,-0.371272,-0.588422,0.209128,-0.340458,-0.204045,0.833681,0.618681,0.618628,1.368644,0.017381,-0.003542,-0.383269,-1.224733,-7.070999e-01,0.691398,1.262048,0.606666,0.851630,-0.888754,0.574812,-0.175084,-0.854898,-0.262266,0.384631,-1.003771,0.348153,-0.810450,0.085401,0.008392,-0.787046,0.969858,0.247312,-1.224742,-7.071051e-01,0.691403,1.262057,0.790456,0.217412,-1.007078,-1.193811,0.013115,0.005503,0.074518,-0.770121,0.160003,0.111942,-0.893193,-0.517030,0.038822,-0.418447,-0.035250,-0.004683,-1.224742,-7.071051e-01,0.691403,1.262057,-0.276332,-0.034668,-0.708004,-1.638159,-0.557960,-0.666563,-0.168524,1.793736,-0.092408,-0.183010,-0.951971,-0.519462,-0.301794,-0.341598,0.101417,0.038849,-1.224742,-7.071051e-01,0.691403,1.262057,-0.107791,0.114886,-0.666708,-0.695143,-0.326052,-0.385286,-0.071976,1.993264,0.067985,0.224368,-1.053052,-0.428888,0.355091,-0.126869,0.589448,0.234604,-1.224742,-7.071051e-01,0.691403,1.262057
35801,-1.143427,-0.683542,-0.035966,-0.532714,-0.114719,-0.596201,-0.748928,-0.579040,-1.366021,-3.660242e-01,0.692272,1.261540,-1.180503,-0.816693,0.392822,-0.957662,-0.008820,-0.045589,-0.544518,-0.609542,-1.366021,-3.660242e-01,0.692272,1.261540,-1.489471,-0.550392,1.592033,0.781970,0.487087,1.213433,-0.162048,0.564968,-1.366021,-3.660242e-01,0.692272,1.261540,-0.200502,-0.979243,-1.053718,0.272888,-0.031083,-0.371636,-0.588572,0.208240,-0.343394,-0.206962,0.831219,0.612352,0.626977,1.380089,0.017834,-0.003351,-0.382917,-1.366012,-3.660218e-01,0.692267,1.261532,0.601899,0.850294,-0.888538,0.574477,-0.176067,-0.857346,-0.262341,0.385855,-1.005503,0.353943,-0.810761,0.085608,0.008895,-0.788788,0.978049,0.247721,-1.366022,-3.660245e-01,0.692272,1.261541,0.785702,0.210365,-1.007561,-1.194966,0.010394,-0.001405,0.073152,-0.761483,0.159526,0.109932,-0.894179,-0.516171,0.041146,-0.418404,-0.035432,-0.001309,-1.366022,-3.660245e-01,0.692272,1.261541,-0.273591,-0.033371,-0.708490,-1.637554,-0.557713,-0.668176,-0.168857,1.789578,-0.092393,-0.181129,-0.952937,-0.518781,-0.300862,-0.342035,0.100919,0.040250,-1.366022,-3.660245e-01,0.692272,1.261541,-0.104359,0.109971,-0.668132,-0.696731,-0.326201,-0.385870,-0.070582,1.989079,0.069398,0.222478,-1.054104,-0.429720,0.356830,-0.124732,0.585207,0.236716,-1.366022,-3.660245e-01,0.692272,1.261541
35802,-1.148577,-0.683542,-0.055424,-0.532714,-0.155739,-0.535362,-0.767511,-0.553431,-1.414209,-2.493397e-13,0.693141,1.261024,-1.183593,-0.694634,0.384331,-0.957662,-0.003904,-0.267059,-0.544518,-0.620292,-1.414209,-2.493397e-13,0.693141,1.261024,-1.489471,-0.550392,1.598521,0.764264,0.384039,1.142495,-0.081036,0.548030,-1.414209,-2.493397e-13,0.693141,1.261024,-0.200500,-0.979626,-1.053333,0.274561,-0.034223,-0.371993,-0.588720,0.207320,-0.346314,-0.209866,0.828761,0.605998,0.635324,1.391524,0.018281,-0.003162,-0.382562,-1.414200,-2.493380e-13,0.693136,1.261016,0.597113,0.848960,-0.888298,0.574147,-0.177087,-0.859806,-0.262415,0.387082,-1.007226,0.359732,-0.811071,0.085813,0.009393,-0.790522,0.986231,0.248134,-1.414210,-2.493399e-13,0.693142,1.261025,0.780925,0.203298,-1.008046,-1.196114,0.007664,-0.008324,0.071779,-0.752808,0.159047,0.107931,-0.895165,-0.515311,0.043487,-0.418363,-0.035593,0.00207

In [31]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(model=gru_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['GRU'])
print(val_pbias['GRU'])

(10229, 137)
2
(415, 168, 137)
(415, 120, 1)
(415, 120, 1)
predictions.shape = (415, 120, 1)
pred_day.shape = (415, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 7.036435551106674
nse3 = [0.59095736]
label_day[predict_day].shape = (5, 1)
mean = 7.012417200064856
0.5909573647730137
4.4672689087622235


In [32]:
val_nse['LSTM'], val_pbias['LSTM'], pred, label = compa(model=multi_lstm_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['LSTM'])
print(val_pbias['LSTM'])

(10229, 137)
2
(415, 168, 137)
(415, 120, 1)
(415, 120, 1)
predictions.shape = (415, 120, 1)
pred_day.shape = (415, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 7.036435551106674
nse3 = [0.50316534]
label_day[predict_day].shape = (5, 1)
mean = 7.012417200064856
0.5031653367245281
8.623713794156636


In [33]:
val_nse['CNN'], val_pbias['CNN'], pred, label = compa(model=multi_conv_model,
                                                      df=val_df,
                                                      plot_col=out_features[0], 
                                                      target_std=target_std, 
                                                      target_mean=target_mean,
                                                      predict_day = 4)

print(val_nse['CNN'])
print(val_pbias['CNN'])

(10229, 137)
2
(415, 168, 137)
(415, 120, 1)
(415, 120, 1)
predictions.shape = (415, 120, 1)
pred_day.shape = (415, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 7.036435551106674
nse3 = [0.3773021]
label_day[predict_day].shape = (5, 1)
mean = 7.012417200064856
0.3773021010308877
2.3457326025188636


In [32]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=train_df,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(35800, 36)
2
(1480, 168, 36)
(1480, 120, 1)
(1480, 120, 1)
predictions.shape = (1480, 120, 1)
pred_day.shape = (1480, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 7.8149443
nse3 = [0.974932]
label_day[predict_day].shape = (5, 1)
mean = 7.814469


0.9749320149421692
0.20658262073993683


In [33]:
val_nse['GRU'], val_pbias['GRU'], pred, label = compa(
    model=gru_model,df=real_df_all,
    plot_col=out_features[0], target_std=target_std, target_mean=target_mean, predict_day = 4)

print()
print()

print(val_nse['GRU'])
print(val_pbias['GRU'])

(51144, 36)
2
(2119, 168, 36)
(2119, 120, 1)
(2119, 120, 1)
predictions.shape = (2119, 120, 1)
pred_day.shape = (2119, 5, 1)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

o1 = 7.7086835
nse3 = [0.95169735]
label_day[predict_day].shape = (5, 1)
mean = 7.7147675


0.9516973495483398
0.9039307832717896


In [34]:
x = np.arange(len(val_nse))
width = 0.35
plt.figure()
plt.title(watershed + '['+start_year+','+end_year+']' + rnn_target_column)
# plt.bar(x, val_pbias.values(), 0.3, label='PBIAS' )
plt.bar(val_nse.keys(), val_nse.values(), 0.3, label='NSE')
# plt.xticks(x,val_nse.keys(), rotation=45)
_ = plt.legend()
plt.show()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …